In [0]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
#!pip install jupyter
#!pip install matplotlib

!git clone https://github.com/tensorflow/models.git

%cd /content/models/research

!protoc object_detection/protos/*.proto --python_out=.

%set_env PYTHONPATH=/content/models/research:/content/models/research/slim

!python object_detection/builders/model_builder_test.py

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-tk is already the newest version (2.7.17-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 25 not upgraded.
Need to get 1,818 kB of archives.
After this operation, 7,688 kB of additional disk space will be used.
Get:1 htt

In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

print('tensorflow version = ',tf.__version__)

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

tensorflow version =  1.15.0


In [0]:
!nvcc -V
!sudo lsb_release -a
!cat /usr/local/cuda/version.txt

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.3 LTS
Release:	18.04
Codename:	bionic
CUDA Version 10.0.130


In [0]:
!pwd

/content/models/research/object_detection


In [0]:
%cd ./object_detection/

from utils import label_map_util

from utils import visualization_utils as vis_util

[Errno 2] No such file or directory: './object_detection/'
/content/models/research/object_detection


In [0]:
# This is needed to display the images.
%matplotlib inline

In [0]:
import os
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

workspace_id = '' # Your code of google-drive sharing

downloaded = drive.CreateFile({'id': workspace_id})
print(downloaded['title'])  
downloaded.GetContentFile(os.path.join('/content', 'Traffic-Light-Classification_colab.zip'))

zip_ref = ZipFile("/content/Traffic-Light-Classification_colab.zip", 'r')
zip_ref.extractall("/content")
zip_ref.close()

Traffic-Light-Classification_colab.zip


In [0]:
cd /content/Traffic-Light-Classification_colab/models

/content/Traffic-Light-Classification_colab


In [0]:
MODEL_NAME = 'ssd_inception_v2_coco_2018_01_28'
# MODEL_NAME = 'ssd_mobilenet_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03'
# MODEL_NAME = 'ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03'

In [0]:
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DOWNLOAD_PATH= '/content/Traffic-Light-Classification_colab/models/'+MODEL_FILE

In [0]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, DOWNLOAD_PATH)
!tar -zxvf MODEL_FILE

In [0]:
!python train.py --logtostderr --train_dir=./final_models/ssd_inception_v2_sim --pipeline_config_path=./config/ssd_inception_v2_coco_simulator.config

Streaming output truncated to the last 5000 lines.
I0213 13:08:36.766707 139831798888320 learning.py:507] global step 17518: loss = 1.5014 (0.568 sec/step)
INFO:tensorflow:global step 17519: loss = 0.9536 (0.613 sec/step)
I0213 13:08:37.381881 139831798888320 learning.py:507] global step 17519: loss = 0.9536 (0.613 sec/step)
INFO:tensorflow:global step 17520: loss = 1.7909 (0.574 sec/step)
I0213 13:08:37.957146 139831798888320 learning.py:507] global step 17520: loss = 1.7909 (0.574 sec/step)
INFO:tensorflow:global step 17521: loss = 1.6347 (0.576 sec/step)
I0213 13:08:38.534996 139831798888320 learning.py:507] global step 17521: loss = 1.6347 (0.576 sec/step)
INFO:tensorflow:global step 17522: loss = 1.9387 (0.580 sec/step)
I0213 13:08:39.116943 139831798888320 learning.py:507] global step 17522: loss = 1.9387 (0.580 sec/step)
INFO:tensorflow:global step 17523: loss = 1.6184 (0.615 sec/step)
I0213 13:08:39.733559 139831798888320 learning.py:507] global step 17523: loss = 1.6184 (0.615

In [0]:
!python export_inference_graph1.14.py --input_type image_tensor --pipeline_config_path ./config/ssd_inception_v2_coco_simulator.config --trained_checkpoint_prefix ./final_models/ssd_inception_v2_sim/model.ckpt-20000 --output_directory final_models/ssd_inc_v2_sim_finetune


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0213 14:15:06.051369 140246047750016 module_wrapper.py:139] From export_inference_graph.py:145: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0213 14:15:06.057844 140246047750016 module_wrapper.py:139] From /content/models/research/object_detection/exporter.py:402: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0213 14:15:06.058155 140246047750016 module_wrapper.py:139] From /content/models/research/object_detection/exporter.py:121: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


W0213 14:15:06.091721 140246047750016

In [0]:
!zip -r /content/ssd_inc_v2_sim.zip /content/Traffic-Light-Classification_colab/final_models/ssd_inc_v2_sim_finetune


  adding: content/Traffic-Light-Classification_colab/final_models/ssd_inc_v2_sim_finetune/ (stored 0%)
  adding: content/Traffic-Light-Classification_colab/final_models/ssd_inc_v2_sim_finetune/saved_model/ (stored 0%)
  adding: content/Traffic-Light-Classification_colab/final_models/ssd_inc_v2_sim_finetune/saved_model/saved_model.pb (deflated 8%)
  adding: content/Traffic-Light-Classification_colab/final_models/ssd_inc_v2_sim_finetune/saved_model/variables/ (stored 0%)
  adding: content/Traffic-Light-Classification_colab/final_models/ssd_inc_v2_sim_finetune/model.ckpt.data-00000-of-00001 (deflated 7%)
  adding: content/Traffic-Light-Classification_colab/final_models/ssd_inc_v2_sim_finetune/model.ckpt.index (deflated 70%)
  adding: content/Traffic-Light-Classification_colab/final_models/ssd_inc_v2_sim_finetune/model.ckpt.meta (deflated 94%)
  adding: content/Traffic-Light-Classification_colab/final_models/ssd_inc_v2_sim_finetune/frozen_inference_graph.pb (deflated 8%)
  adding: content/